In [89]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import r2_score
from skorch import NeuralNetRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
# load training data
Xtr_loadpath = 'Xtr.csv'
Xts_loadpath = 'Xts.csv'
ytr_loadpath = 'ytr.csv'

Xtr = np.loadtxt(Xtr_loadpath, delimiter=",")
Xts = np.loadtxt(Xts_loadpath, delimiter=",")
ytr = np.loadtxt(ytr_loadpath, delimiter=",")

In [3]:
# standardize the training data
Xtr_mean = np.mean(Xtr,axis=0)
Xtr_std = np.std(Xtr,axis=0)
Xtr_standardized = ((Xtr-Xtr_mean[None,:])/Xtr_std[None,:]) # revise this line as needed
Xts_standardized = ((Xts-Xtr_mean[None,:])/Xtr_std[None,:]) # revise this line as needed

# save the standardized training data
Xtr_savepath = 'Xtr_pytorch.csv'
Xts_savepath = 'Xts_pytorch.csv'
ytr_savepath = 'ytr_pytorch.csv'
yts_hat_savepath = 'yts_hat_pytorch.csv'

np.savetxt(Xtr_savepath, Xtr_standardized, delimiter=",")
np.savetxt(Xts_savepath, Xts_standardized, delimiter=",")
np.savetxt(ytr_savepath, ytr, delimiter=",")

In [4]:
# Convert the numpy arrays to PyTorch tensors
Xtr_torch = torch.Tensor(Xtr_standardized)
ytr_torch = torch.Tensor(ytr)

batch_size = 100  # size of each batch

# Create a training Dataset
train_ds = torch.utils.data.TensorDataset(Xtr_torch, ytr_torch)
# Creates a training DataLoader from this Dataset
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True) 


In [108]:
nin = Xtr.shape[1]
nout = 1
#nh = 256

model = nn.Sequential(
    nn.Linear(nin, 32),
    nn.ReLU(),
    #nn.Linear(64, 32, bias=False),
    #nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 8),
    nn.ReLU(),
    nn.Linear(8, nout)
)

print(str(model))

Sequential(
  (0): Linear(in_features=26, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=16, bias=True)
  (3): ReLU()
  (4): Linear(in_features=16, out_features=8, bias=True)
  (5): ReLU()
  (6): Linear(in_features=8, out_features=1, bias=True)
)


In [109]:
# ## Hyper Parameter Gird Search Cross-Validation

# import sklearn.model_selection 

# # construct repeated k-fold cross-val object
# nfold = 10
# nrep = 5
# rkf = sklearn.model_selection.RepeatedKFold(n_splits=nfold,
#                                             n_repeats=nrep, random_state=1)

# # Define your PyTorch neural network as a class
# class gscvNet(nn.Module):
#     def __init__(self):
#         super(gscvNet,self).__init__()
#         self.relu = nn.ReLU()
#         self.hl1 = nn.Linear(26, 32)
#         self.hl2 = nn.Linear(32, 16)
#         self.hl3 = nn.Linear(16, 8)
#         self.hl4 = nn.Linear(8, 1)
        
#     def forward(self,x):        
#         x = self.hl1(x)
#         x = self.relu(x)
#         x = self.hl2(x)
#         x = self.relu(x)
#         x = self.hl3(x)
#         x = self.relu(x)
#         out = self.hl4(x)
#         #return x        
#         return out

# # Create a dictionary of hyperparameters and their possible values
# ##nh1_space = np.linspace(20,60,1)
# ##nh2_space = np.linspace(10,40,1)
# ##nh3_space = np.linspace(5,20,1)
# criterion_space = [torch.nn.MSELoss, torch.nn.L1Loss, torch.nn.HuberLoss]

# param_grid = {
    
#     'criterion': criterion_space
#     #'optimizer': [optim.Adam, optim.SGD]
#     #'optimizer_lr': [0.01, 0.001],
# }

# # Create a PyTorch neural network object using the above-defined class
# cv_net = NeuralNetRegressor(
#     gscvNet,
#     max_epochs=25,
#     #criterion=nn.MSELoss,
# )

# # Create a GridSearchCV object using the neural network and the hyperparameter dictionary
# gscv = GridSearchCV(cv_net, param_grid, cv=rkf, scoring='r2')

# # Fit the GridSearchCV object to your data
# gscv.fit(Xtr_standardized.dtype(torch.long), ytr.reshape(-1, 1).dtype(torch.long))

# # Print the best hyperparameters and the corresponding R-squared score
# print(gs.best_params_, gs.best_score_)

In [112]:
# Choosing the optimizer and loss function

opt = optim.Adam(model.parameters(), lr=0.01)
#criterion = nn.MSELoss()
criterion = nn.HuberLoss(reduction='mean', delta=0.1)

In [113]:
# training the model
num_epoch = 100

a_tr_loss = np.zeros([num_epoch])
a_tr_Rsq = np.zeros([num_epoch])

for epoch in range(num_epoch):

    model.train() # put model in training mode
    batch_loss_tr = []
    batch_Rsq_tr = []
    # iterate over training set
    for train_iter, data in enumerate(train_loader):
        x_batch,y_batch = data
        
        y_batch = y_batch.view(-1,1)
        #y_batch = y_batch.type(torch.long)
        
        out = model(x_batch)
        # Compute Loss
        loss = criterion(out,y_batch.type(torch.float))
        batch_loss_tr.append(loss.item())
        # Compute R-square
        Rsq = r2_score(y_batch.detach().numpy(), out.detach().numpy())
        batch_Rsq_tr.append(Rsq.item())
        # Compute gradients using back propagation
        opt.zero_grad()
        loss.backward()
        # Take an optimization 'step'
        opt.step()
        
    a_tr_loss[epoch] = np.mean(batch_loss_tr) # Compute average loss over epoch
    a_tr_Rsq[epoch] = np.mean(batch_Rsq_tr)
    print('Epoch: {0:2d}   Train Loss: {1:.3f}   '.format(epoch+1, a_tr_loss[epoch])
         + 'R^2: {0:.3f}   '.format(a_tr_Rsq[epoch])
         )
        

Epoch:  1   Train Loss: 0.327   R^2: 0.338   
Epoch:  2   Train Loss: 0.327   R^2: 0.333   
Epoch:  3   Train Loss: 0.324   R^2: 0.346   
Epoch:  4   Train Loss: 0.325   R^2: 0.356   
Epoch:  5   Train Loss: 0.324   R^2: 0.352   
Epoch:  6   Train Loss: 0.322   R^2: 0.362   
Epoch:  7   Train Loss: 0.322   R^2: 0.348   
Epoch:  8   Train Loss: 0.322   R^2: 0.352   
Epoch:  9   Train Loss: 0.322   R^2: 0.356   
Epoch: 10   Train Loss: 0.319   R^2: 0.370   
Epoch: 11   Train Loss: 0.320   R^2: 0.369   
Epoch: 12   Train Loss: 0.319   R^2: 0.360   
Epoch: 13   Train Loss: 0.321   R^2: 0.363   
Epoch: 14   Train Loss: 0.320   R^2: 0.351   
Epoch: 15   Train Loss: 0.322   R^2: 0.371   
Epoch: 16   Train Loss: 0.318   R^2: 0.384   
Epoch: 17   Train Loss: 0.318   R^2: 0.368   
Epoch: 18   Train Loss: 0.320   R^2: 0.371   
Epoch: 19   Train Loss: 0.317   R^2: 0.375   
Epoch: 20   Train Loss: 0.318   R^2: 0.368   
Epoch: 21   Train Loss: 0.316   R^2: 0.383   
Epoch: 22   Train Loss: 0.317   R^

In [115]:
# # save the model: you must use the .pth format for pytorch models!
# model_savepath = 'model.pth'

# # To save a PyTorch model, we first pass an input through the model, 
# # and then save the "trace". 
# # For this purpose, we can use any input. 
# # We will create a random input with the proper dimension.
# x = torch.randn(26) # random input
# x = x[None,:] # add singleton batch index
# with torch.no_grad():
#     traced_cell = torch.jit.trace(model, (x))

# # Now we save the trace
# torch.jit.save(traced_cell, model_savepath)

In [116]:
# # generate kaggle submission file using the validation script
# !python {"validation.py " + model_savepath + " --Xts_path " + Xts_savepath + " --Xtr_path " + Xtr_savepath + " --yts_hat_path " + yts_hat_savepath } 

training R2 =  0.44558000919820573
test target predictions saved in yts_hat_pytorch.csv
